In [1]:
import pandas as pd
import numpy as np
from configs import ROOT_DIR
from utils.transformer import standardize_data, tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,classification_report


In [4]:

dataset_file_path = ROOT_DIR + '\outputs\comments\dienthoai\\all.xlsx'
df = pd.read_excel(dataset_file_path)
comments = df.iloc[:, 5]
rates = df.iloc[:, 8]

comments = comments.apply(standardize_data)
comments = comments.apply(tokenizer)


print(len(comments))


<ipython-input-4-eb4aa4cd8865>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(dataset_file_path)
4614


In [5]:
def get_model():
    # model = SVC(kernel='linear', C=1)
    # model = LogisticRegression(random_state=0)
    model = MLPClassifier(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000)
    return model


In [6]:
# X_train, X_test, y_train, y_test = train_test_split(
#     comments, rates, test_size=0.2, random_state=42)

X = comments
y =rates

emb = TfidfVectorizer(min_df=5, max_df=1.0,
                        max_features=3000, sublinear_tf=True)
emb.fit(X)
X = emb.transform(X)
print(type(X))
print(X.shape)
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)
 
# K-fold Cross Validation model evaluation
fold_idx = 1
accuracy_list = []
for train_ids, val_ids in kfold.split(X, y):
    
    model = get_model()
    
    
    print("Bắt đầu train Fold ", fold_idx)

    # Train model
    model.fit(X[train_ids], y[train_ids])
 
    # Test và in kết quả
    y_pred = model.predict(X[val_ids])
    scores = accuracy_score(y[val_ids],y_pred)
    print("Đã train xong Fold ", fold_idx)
    print(scores)
    print(classification_report(y[val_ids], y_pred, target_names=['️️   ⭐️  ','   ⭐️⭐  ','  ⭐️⭐️⭐️ ',' ⭐️⭐️⭐️⭐️ ','⭐️⭐️⭐️⭐️⭐️']))
    accuracy_list.append(scores * 100)
    
    # Sang Fold tiếp theo
    fold_idx = fold_idx + 1

# In kết quả tổng thể
print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
    print(f'> Fold {i+1} - Accuracy: {accuracy_list[i]}%')
 
print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')



<class 'scipy.sparse.csr.csr_matrix'>
(4614, 1127)
Bắt đầu train Fold  1
Đã train xong Fold  1
0.7009750812567714
              precision    recall  f1-score   support

   ️️   ⭐️         0.54      0.52      0.53        73
       ⭐️⭐         0.06      0.04      0.05        27
     ⭐️⭐️⭐️        0.17      0.13      0.15        62
   ⭐️⭐️⭐️⭐️        0.22      0.22      0.22       103
  ⭐️⭐️⭐️⭐️⭐️       0.84      0.88      0.86       658

    accuracy                           0.70       923
   macro avg       0.37      0.36      0.36       923
weighted avg       0.68      0.70      0.69       923

Bắt đầu train Fold  2
Đã train xong Fold  2
0.6803900325027086
              precision    recall  f1-score   support

   ️️   ⭐️         0.44      0.46      0.45        65
       ⭐️⭐         0.08      0.08      0.08        25
     ⭐️⭐️⭐️        0.17      0.14      0.16        56
   ⭐️⭐️⭐️⭐️        0.26      0.25      0.26       122
  ⭐️⭐️⭐️⭐️⭐️       0.84      0.85      0.84       655

    accu

In [6]:

emb = TfidfVectorizer(min_df=5, max_df=0.8,
                      max_features=3000, sublinear_tf=True)
emb.fit(X_train)
X_train = emb.transform(X_train)
X_test = emb.transform(X_test)

# cv = CountVectorizer(max_df=0.85)
# word_count_vector = cv.fit_transform(pre_datasets)
# print(word_count_vector)
# print(list(cv.vocabulary_.keys())[:10])

# tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
# tfidf_transformer.fit(word_count_vector)

In [7]:
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(X_train, y_train)

model_SVM = SVC(kernel='linear', C=1)
model_SVM.fit(X_train, y_train)

y_pred_1 = model_LinearRegression.predict(X_test)
y_pred_2 = model_SVM.predict(X_test)

y_pred_1 = [(round(y) if y > 1 and y < 5 else (1 if y < 1 else 5))
            for y in y_pred_1]

print("Linear Regression : {} %".format(accuracy_score(y_test, y_pred_1)))
print("SVM : {} %".format(accuracy_score(y_test, y_pred_2)))

Linear Regression : 0.5882990249187432 %
SVM : 0.7291440953412784 %
